In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from collections import Counter
import re

## EDA

## Base Question 1: What are the end-to-end travel times for different bus routes?

To address the research question, our team initiated a preliminary investigation to identify suitable datasets related to MBTA Bus Routes and travel times. Subsequent research led us to discover a dataset containing bus arrival and departure events. We proceeded to download and meticulously analyze this dataset, determining that it contained sufficient data to effectively address the initial and secondary research inquiries.

Since we knew the timeframe focus of this project was January 1, 2022 - January 31, 2022 (mentioned in the overview document), we specifically downloaded the [MBTA Bus Arrival Departure Times 2022](https://mbta-massdot.opendata.arcgis.com/datasets/mbta-bus-arrival-departure-times-2022/about).

In [6]:
q1_file_path = "./data/question-1/raw/1.csv" # 1 denoting January 2022
df = pd.read_csv(q1_file_path)

We performed basic EDA on the dataset.

In [7]:
df.shape

(2348823, 13)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348823 entries, 0 to 2348822
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   service_date       object 
 1   route_id           object 
 2   direction_id       object 
 3   half_trip_id       float64
 4   stop_id            int64  
 5   time_point_id      object 
 6   time_point_order   int64  
 7   point_type         object 
 8   standard_type      object 
 9   scheduled          object 
 10  actual             object 
 11  scheduled_headway  float64
 12  headway            float64
dtypes: float64(3), int64(2), object(8)
memory usage: 233.0+ MB


In [9]:
# Checking if there are any duplicate data in our dataset
df.duplicated().sum()

0

In [10]:
# Checking if there are any null values in our dataset
df.isnull().sum()

service_date               0
route_id                   0
direction_id               0
half_trip_id             213
stop_id                    0
time_point_id              0
time_point_order           0
point_type                 0
standard_type              0
scheduled                  0
actual                180264
scheduled_headway    1196318
headway              1418416
dtype: int64

To work with the time data, we decided to convert our `scheduled` and `actual` data column to a pandas datetime object, making comparisons and operations much more easier to work with.

In [11]:
df['scheduled'] = pd.to_datetime(df['scheduled'])
df['actual'] = pd.to_datetime(df['actual'])

Knowing that we have data from January 1 to January 31 of 2022, we decided to find the inbound and outbound travel times for each date as weekday, weekend, or holiday travel times could differ.

We created a helper function that does the necessary operations to group the routes together and find the raw, average, and median end-to-end times of each route's inbound and outbound times.

In [22]:
def process_end_to_end_time(date):
    date_pattern = r'^\d{4}-\d{2}-\d{2}$'
    if not re.match(date_pattern, date):
        raise "Incorrect input."
        
    # Get schedules of given date
    df_type_schedule = df[df["standard_type"] == "Schedule"]
    # df_type_schedule = df_type_schedule[df_type_schedule["service_date"] == "2022-01-03"]
    df_type_schedule = df_type_schedule[df_type_schedule["service_date"] == date]

    # Filter by startpoint and endpoint
    df_start_end = df_type_schedule[(df_type_schedule["point_type"] == "Startpoint") | (df_type_schedule["point_type"] == "Endpoint")]

    # Filter for "Startpoint" and "Endpoint" entries
    startpoints = df[df["point_type"] == "Startpoint"]
    endpoints = df[df["point_type"] == "Endpoint"]
    
    # Group Startpoints and Endpoints by 'half_trip_id'
    startpoint_groups = startpoints.groupby("half_trip_id")
    endpoint_groups = endpoints.groupby("half_trip_id")
    
    # Calculate the end-to-end travel time for each 'half_trip_id'
    end_to_end_travel_times = (endpoint_groups["actual"].max() - startpoint_groups["actual"].min()).dt.total_seconds() / 60
    
    # Merge 'half_trip_id' and 'end-to-end travel times' into a new DataFrame
    end_to_end_df = pd.DataFrame({'half_trip_id': end_to_end_travel_times.index, 'end_to_end_travel_time': end_to_end_travel_times.values})
    
    # Extract the start and end times for each 'half_trip_id'
    start_times = startpoint_groups["actual"].min()
    end_times = endpoint_groups["actual"].max()
    
    # Add the start and end times to the 'end_to_end_df'
    end_to_end_df["start_time"] = start_times.loc[end_to_end_df["half_trip_id"].values].values
    end_to_end_df["end_time"] = end_times.loc[end_to_end_df["half_trip_id"].values].values
    
    # Merge the 'route_id' and 'direction_id' from the original DataFrame
    end_to_end_df = end_to_end_df.merge(df[['half_trip_id', 'route_id', 'direction_id']], on='half_trip_id', how='left')
    
    # Reorder the columns for better readability
    end_to_end_df = end_to_end_df[['half_trip_id', 'route_id', 'direction_id', 'start_time', 'end_time', 'end_to_end_travel_time']]

    unique_end_to_end = end_to_end_df.groupby(['half_trip_id', 'route_id', 'direction_id'], as_index=False).agg({
        'start_time': 'first',
        'end_time': 'first',
        'end_to_end_travel_time': 'mean'
    })

    unique_end_to_end = unique_end_to_end.dropna()
    output_file_path = f'./data/question-1/processed/raw/{date}.csv'  # Replace with your desired file path
    unique_end_to_end.to_csv(output_file_path, index=False)

    mean_end_to_end = unique_end_to_end.groupby(['route_id', 'direction_id'], as_index=False)['end_to_end_travel_time'].mean()
    output_file_path = f'./data/question-1/processed/avg/{date}.csv'  # Replace with your desired file path
    mean_end_to_end.to_csv(output_file_path, index=False)
        
    median_end_to_end = unique_end_to_end.groupby(['route_id', 'direction_id'], as_index=False)['end_to_end_travel_time'].median()
    output_file_path = f'./data/question-1/processed/median/{date}.csv'  # Replace with your desired file path
    median_end_to_end.to_csv(output_file_path, index=False)

In [23]:
process_end_to_end_time("2022-01-01")

Here is an example of the raw, average and median end-to-end travel times of each route.

In [24]:
df_jan1 = pd.read_csv("./data/question-1/processed/raw/2022-01-01.csv")
df_jan1_avg = pd.read_csv("./data/question-1/processed/avg/2022-01-01.csv")
df_jan1_median = pd.read_csv("./data/question-1/processed/median/2022-01-01.csv")

In [25]:
df_jan1.head()

,half_trip_id,route_id,direction_id,start_time,end_time,end_to_end_travel_time
0,54136531.0,21,Outbound,1900-01-01 20:03:18,1900-01-01 20:15:58,12.666667
1,54136532.0,26,Inbound,1900-01-01 20:25:55,1900-01-01 20:36:01,10.100000
2,54136533.0,26,Outbound,1900-01-01 20:36:33,1900-01-01 20:45:40,9.116667
3,54136534.0,21,Inbound,1900-01-01 20:58:19,1900-01-01 21:12:12,13.883333
4,54136535.0,42,Inbound,1900-01-01 21:27:30,1900-01-01 21:38:00,10.500000


In [26]:
df_jan1_avg.head()

,route_id,direction_id,end_to_end_travel_time
0,01,Inbound,30.473510
1,01,Outbound,30.899495
2,04,Inbound,25.630918
3,04,Outbound,18.729442
4,07,Inbound,14.421404


In [27]:
df_jan1_median.head()

,route_id,direction_id,end_to_end_travel_time
0,01,Inbound,30.108333
1,01,Outbound,30.533333
2,04,Inbound,21.850000
3,04,Outbound,16.266667
4,07,Inbound,14.266667


Then, we proceeded to generate the processed data for each day in January.

In [28]:
for day in range(1, 32):
    # Format the day as a two-digit string
    formatted_day = f"{day:02}"

    # Create the date string with the formatted day
    date = f"2022-01-{formatted_day}"

    # Call your function with the date
    process_end_to_end_time(date)